In [39]:
# Stats on Ca2+ imaging with miniscope and Osc

#######################################################################################
                            # Define Experiment type #
#######################################################################################

DrugExperiment=0 #if CGP Experiment
#DrugExperiment=0 #if Baseline Experiment
suffix='_AH_FINAL'

choosed_folder='Osc_2024_08_04_10_35_33_AH_FINAL' if DrugExperiment else 'Osc_2024_08_04_09_24_52_AH_FINAL'
PrefVigExcel_file = 'AVG_VigSt_2024-08-04_13_06_25_AB_FINAL' if DrugExperiment else 'AVG_VigSt_2024-08-04_10_42_04_AB_FINAL'
                
#######################################################################################
                                # Load packages #
#######################################################################################

import statsmodels.api as sm
import quantities as pq
import numpy as np
import math 
import neo
import json
from pathlib import Path
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
import pickle
import os
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import zscore
from scipy.stats import sem
from datetime import datetime
import shutil
from scipy import stats
from scipy.stats import t
from scipy.stats import norm

import warnings
warnings.filterwarnings("ignore")


def column_with_max_single_per_row(row):
    max_val = row.max()  # Find the max value in the row
    max_columns = row == max_val  # Boolean Series of columns with max value
    
    if max_columns.sum() > 1:
        return 'NoPref'  # More than one column has the max value
    else:
        return max_columns.idxmax()  # Return the name of the column with max value

########################################################################
        # SCRIPT 27AB_GrandAverages&Stats_for_Osc
########################################################################

# Specify the directory containing the Excel files
InitialDirectory = "//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/CGP/AB_Analysis" if DrugExperiment else "//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording_ABmodified/AB_Analysis"
directory= f'{InitialDirectory}/{choosed_folder}'

# Get the current date and time
FolderNameSave=str(datetime.now())[:19]
FolderNameSave = FolderNameSave.replace(" ", "_").replace(".", "_").replace(":", "_")
destination_folder= f"//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/CGP/AB_Analysis/AVG_Osc_{FolderNameSave}{suffix}/" if DrugExperiment else f"//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording_ABmodified/AB_Analysis/AVG_Osc_{FolderNameSave}{suffix}/"
os.makedirs(destination_folder)
folder_to_save=Path(destination_folder)

# Copy the script file to the destination folder
source_script = "C:/Users/Manip2/SCRIPTS/CodePythonAudrey/CodePythonAurelie/HayLabAnalysis/python/25_28_GrandAverage&Stats_for_DownStatesSpdl&SWR_FullAuto.ipynb"
destination_file_path = f"{destination_folder}/25_28_GrandAverage&Stats_for_DownStatesSpdl&SWR_FullAuto.txt"
shutil.copy(source_script, destination_file_path)

NrSubtypeList=['L2&3', 'L1']
CTX=['PFC_', 'S1_', 'S1PFC_']
Coupling=['', 'UnCoupled', 'Coupled']
OscList=['Spdl', 'SWR']#, 'DS']
Drugs= ['Baseline', 'CGP'] if DrugExperiment else ['Baseline']

for o, Osc in enumerate(OscList): 
    
    print(Osc, 'oscillations analysis...')


    AllOscStatut=pd.DataFrame()
    AllOscDuration=pd.DataFrame()
    AllOscCoupling=pd.DataFrame()
    AllGlobalSpindle=pd.DataFrame()
    AllOscStartLocation=pd.DataFrame()
    
    AllOscStatutS1=pd.DataFrame()
    AllOscDurationS1=pd.DataFrame()
    AllOscCouplingS1=pd.DataFrame()

    AllOscStatutPFC=pd.DataFrame()
    AllOscDurationPFC=pd.DataFrame()
    AllOscCouplingPFC=pd.DataFrame()

    AllOscStatutS1PFC=pd.DataFrame()
    AllOscDurationS1PFC=pd.DataFrame()
    AllOscCouplingS1PFC=pd.DataFrame()

    for NrSubtype in NrSubtypeList: 
        
        print('... for', NrSubtype, 'neurons...')
        
        # Initialize an empty list to store the dataframes
        dfs = []
        df=[]
        dfs2 = []
        df2=[]
        dfs2_per_sheet = {}
        dfs3 = []
        df3=[]
        dfs3_per_sheet = {}
        dfs4 = []
        df4=[]
        dfs4_per_sheet = {}
        filtered_df=[]

        if NrSubtype=='L1':
            MiceList=['BlackLinesOK', 'BlueLinesOK', 'GreenDotsOK', 'GreenLinesOK', 'RedLinesOK']
        else:
            MiceList=['Purple', 'ThreeColDotsOK', 'ThreeBlueCrossesOK']

        nametofind=f'{Osc}_Global'
        nametofind2=f'{Osc}_CaPSTH'
        nametofind3=f'{Osc}_SpPSTH'
        #nametofind4=f'{OscillationList[o]}_{Cortex}_SpikeSum'

        # Recursively traverse the directory structure
        for root, _, files in os.walk(directory):
            for filename in files:
                # Check if the file is an Excel file and contains the specified name
                if filename.endswith('.xlsx') and nametofind in filename:
                    if any(name in filename for name in MiceList):  
                        # Construct the full path to the file
                        filepath = os.path.join(root, filename)
                        # Read the Excel file into a dataframe and append it to the list
                        df = pd.read_excel(filepath, index_col=0)
                        dfs.append(df)
                        print(filename)
                if filename.endswith('.xlsx') and nametofind2 in filename: 
                    if any(name in filename for name in MiceList): 
                        # Construct the full path to the file
                        filepath = os.path.join(root, filename)
                        # Read the Excel file into a dataframe and append it to the list
                        excel_data = pd.read_excel(filepath, sheet_name=None, index_col=0, header=None)           
                        for sheet_name, df2 in excel_data.items():
                            if len(df2)>0:
                                if sheet_name in dfs2_per_sheet:                                       
                                    updated_matrix = pd.concat([dfs2_per_sheet[sheet_name], df2], ignore_index=False, axis=0)                    
                                    dfs2_per_sheet[sheet_name] = updated_matrix    
                                else:
                                    dfs2_per_sheet[sheet_name] = df2  #one average trace per unique unit, len(df2)==nb unit recorded for that mouse
                        print(filename)
                if filename.endswith('.xlsx') and nametofind3 in filename: 
                    if any(name in filename for name in MiceList): 
                        # Construct the full path to the file
                        filepath = os.path.join(root, filename)
                        # Read the Excel file into a dataframe and append it to the list
                        excel_data = pd.read_excel(filepath, sheet_name=None, index_col=0, header=None)           
                        for sheet_name, df3 in excel_data.items():
                            if sheet_name in dfs3_per_sheet:   
                                updated_matrix = pd.concat((dfs3_per_sheet[sheet_name],df3), ignore_index=False, axis=0)                
                                dfs3_per_sheet[sheet_name] = updated_matrix                    
                            else:                    
                                dfs3_per_sheet[sheet_name] = df3 #one average trace per unique unit, len(df3)==nb unit recorded for that mouse
                        print(filename)

        ###########################################################################
                                    ##### GLOBAL #####
        ###########################################################################

        # Concatenate all dataframes into a single dataframe
        try: combined_df = pd.concat(dfs, ignore_index=True)
        except: combined_df =pd.DataFrame(dfs)

        combined_df['Unique_Unit'] = combined_df['Unique_Unit'].astype(str)
        combined_df['UnitNumber'] = combined_df['UnitNumber'].astype(str)
        combined_df['UnitValue'] = combined_df['UnitValue'].astype(str)

        combined_df[f'{Osc}Statut'] = combined_df[f'{Osc}Statut'].astype(str)

        combined_df['Unit_ID'] = combined_df['Mice'] + combined_df['Unique_Unit']
        combined_df['Session_ID'] = combined_df['Mice'] + combined_df['Session'].astype(str)

        unique_count = combined_df['Unit_ID'].nunique()
        print(unique_count, f'{NrSubtype} neurons recorded') 

        # Remove non defined Unique Units 
        combined_df = combined_df[combined_df['Unique_Unit'] != '[]']
        combined_df = combined_df.dropna(subset=['Unique_Unit'])
        unique_count = combined_df['Unit_ID'].nunique()
        print(unique_count, f'{NrSubtype} neurons in the cross-registration') 
        
        combined_df[f'{Osc}_ID'] = combined_df['Mice'] + combined_df['Session'] + combined_df[f'{Osc}Number'].astype(str)
        
        unique_count = combined_df[f'{Osc}_ID'].nunique()
        print(unique_count, f'{Osc} recorded in total')

        #######################
        # Propreties Osc
        #######################
        filenameOut = f'{folder_to_save}/{Osc}Propreties.xlsx'
        writer = pd.ExcelWriter(filenameOut)
        combined_dfOsc = combined_df.drop_duplicates(subset=f'{Osc}_ID', keep='first')

        if Osc== 'Spdl':
            
            OscStatut = pd.crosstab(index=combined_dfOsc['Session_ID'],columns=[combined_dfOsc['Drug'], combined_dfOsc[f'{Osc}Statut'], combined_dfOsc[f'GlobalSpindle']])
            AllOscStatut=pd.concat([AllOscStatut, OscStatut], axis=0)
            OscDuration = combined_dfOsc.pivot_table(index='Session_ID', columns=[combined_dfOsc['Drug'], combined_dfOsc[f'{Osc}Statut'], combined_dfOsc[f'GlobalSpindle']], values=f'{Osc}Duration', aggfunc='mean')
            AllOscDuration=pd.concat([AllOscDuration, OscDuration], axis=0)
            OscCoupling = pd.crosstab(index=combined_dfOsc['Session_ID'],columns=[combined_dfOsc['Drug'], combined_dfOsc[f'SWR_inside_Spdl'], combined_dfOsc[f'{Osc}Statut'], combined_dfOsc[f'GlobalSpindle']])
            AllOscCoupling=pd.concat([AllOscCoupling, OscCoupling], axis=0)
            
            GlobalSpindle = pd.crosstab(index=combined_dfOsc['Session_ID'],columns=[combined_dfOsc['Drug'], combined_dfOsc[f'{Osc}StartLocation'] , combined_dfOsc[f'GlobalSpindle']])
            AllGlobalSpindle=pd.concat([AllGlobalSpindle, GlobalSpindle], axis=0)
            OscStartLocation = pd.crosstab(index=combined_dfOsc['Session_ID'],columns=[combined_dfOsc['Drug'], combined_dfOsc[f'{Osc}StartLocation']])
            AllOscStartLocation=pd.concat([AllOscStartLocation, OscStartLocation], axis=0)
        else: 
            OscStatut = pd.crosstab(index=combined_dfOsc['Session_ID'],columns=[combined_dfOsc['Drug'], combined_dfOsc[f'{Osc}Statut']])
            AllOscStatut=pd.concat([AllOscStatut, OscStatut], axis=0)
            OscDuration = combined_dfOsc.pivot_table(index='Session_ID', columns=[combined_dfOsc['Drug'], combined_dfOsc[f'{Osc}Statut']], values=f'{Osc}Duration', aggfunc='mean')
            AllOscDuration=pd.concat([AllOscDuration, OscDuration], axis=0)
            OscCoupling = pd.crosstab(index=combined_dfOsc['Session_ID'],columns=[combined_dfOsc['Drug'], combined_dfOsc[f'SWR_inside_Spdl'], combined_dfOsc[f'{Osc}Statut']])
            AllOscCoupling=pd.concat([AllOscCoupling, OscCoupling], axis=0)
            
            """
            #S1 Spdl
            combined_dfOscS1 = combined_dfOsc[(combined_dfOsc[f'{Osc}StartLocation'] == 'S1') & (combined_dfOsc[f'GlobalSpindle'] == 'Local')]
            OscStatut = pd.crosstab(index=combined_dfOscS1['Session_ID'],columns=[combined_dfOscS1['Drug'], combined_dfOscS1[f'{Osc}Statut']])
            AllOscStatutS1=pd.concat([AllOscStatutS1, OscStatut], axis=0)
            OscDuration = combined_dfOscS1.pivot_table(index='Session_ID', columns='Drug', values=f'{Osc}Duration', aggfunc='mean')
            AllOscDurationS1=pd.concat([AllOscDurationS1, OscDuration], axis=0)
            OscCoupling = pd.crosstab(index=combined_dfOscS1['Session_ID'],columns=[combined_dfOscS1['Drug'], combined_dfOscS1[f'SWR_inside_Spdl']])
            AllOscCouplingS1=pd.concat([AllOscCouplingS1, OscCoupling], axis=0)

            #PFC Spdl
            combined_dfOscPFC = combined_dfOsc[(combined_dfOsc[f'{Osc}StartLocation'] == 'PFC') & (combined_dfOsc[f'GlobalSpindle'] == 'Local')]
            OscStatut = pd.crosstab(index=combined_dfOscPFC['Session_ID'],columns=[combined_dfOscPFC['Drug'], combined_dfOscPFC[f'{Osc}Statut']])
            AllOscStatutPFC=pd.concat([AllOscStatutPFC, OscStatut], axis=0)
            OscDuration = combined_dfOscPFC.pivot_table(index='Session_ID', columns='Drug', values=f'{Osc}Duration', aggfunc='mean')
            AllOscDurationPFC=pd.concat([AllOscDurationPFC, OscDuration], axis=0)
            OscCoupling = pd.crosstab(index=combined_dfOscPFC['Session_ID'],columns=[combined_dfOscPFC['Drug'], combined_dfOscPFC[f'SWR_inside_Spdl']])
            AllOscCouplingPFC=pd.concat([AllOscCouplingPFC, OscCoupling], axis=0)

            #S1PFC Spdl
            combined_dfOscS1PFC=combined_dfOsc[combined_dfOsc[f'GlobalSpindle'] == 'Global']
            OscStatut = pd.crosstab(index=combined_dfOscS1PFC['Session_ID'],columns=[combined_dfOscS1PFC['Drug'], combined_dfOscS1PFC[f'{Osc}Statut']])
            AllOscStatutS1PFC=pd.concat([AllOscStatutS1PFC, OscStatut], axis=0)
            OscDuration = combined_dfOscS1PFC.pivot_table(index='Session_ID', columns='Drug', values=f'{Osc}Duration', aggfunc='mean')
            AllOscDurationS1PFC=pd.concat([AllOscDurationS1PFC, OscDuration], axis=0)
            OscCoupling = pd.crosstab(index=combined_dfOscS1PFC['Session_ID'],columns=[combined_dfOscS1PFC['Drug'], combined_dfOscS1PFC[f'SWR_inside_Spdl']])
            AllOscCouplingS1PFC=pd.concat([AllOscCouplingS1PFC, OscCoupling], axis=0)
            """


    AllOscStatut.to_excel(writer, sheet_name=f'CouplingStatut')
    AllOscDuration.to_excel(writer, sheet_name=f'MeanDuration')
    AllOscCoupling.to_excel(writer, sheet_name=f'SWR_inside_Spdl')    
    if Osc== 'Spdl':
        AllGlobalSpindle.to_excel(writer, sheet_name=f'GlobalSpindle')
        AllOscStartLocation.to_excel(writer, sheet_name=f'StartLocation')
        """
        AllOscStatutS1.to_excel(writer, sheet_name=f'S1_CouplingStatut')
        AllOscDurationS1.to_excel(writer, sheet_name=f'S1_MeanDuration')
        AllOscCouplingS1.to_excel(writer, sheet_name=f'S1_SWR_inside_Spdl') 
        
        AllOscStatutPFC.to_excel(writer, sheet_name=f'PFC_CouplingStatut')
        AllOscDurationPFC.to_excel(writer, sheet_name=f'PFC_MeanDuration')
        AllOscCouplingPFC.to_excel(writer, sheet_name=f'PFC_SWR_inside_Spdl') 
        
        AllOscStatutS1PFC.to_excel(writer, sheet_name=f'S1PFC_CouplingStatut')
        AllOscDurationS1PFC.to_excel(writer, sheet_name=f'S1PFC_MeanDuration')
        AllOscCouplingS1PFC.to_excel(writer, sheet_name=f'S1PFC_SWR_inside_Spdl')
        """
    writer.close()

Spdl oscillations analysis...
... for L2&3 neurons...
Spdl_CaPSTH_Purple.xlsx
Spdl_CaPSTH_ThreeBlueCrossesOK.xlsx
Spdl_CaPSTH_ThreeColDotsOK.xlsx
Spdl_Global_Purple.xlsx
Spdl_Global_ThreeBlueCrossesOK.xlsx
Spdl_Global_ThreeColDotsOK.xlsx
Spdl_SpPSTH_Purple.xlsx
Spdl_SpPSTH_ThreeBlueCrossesOK.xlsx
Spdl_SpPSTH_ThreeColDotsOK.xlsx
339 L2&3 neurons recorded
338 L2&3 neurons in the cross-registration
1216 Spdl recorded in total
... for L1 neurons...
Spdl_CaPSTH_BlackLinesOK.xlsx
Spdl_CaPSTH_BlueLinesOK.xlsx
Spdl_CaPSTH_GreenDotsOK.xlsx
Spdl_CaPSTH_GreenLinesOK.xlsx
Spdl_CaPSTH_RedLinesOK.xlsx
Spdl_Global_BlackLinesOK.xlsx
Spdl_Global_BlueLinesOK.xlsx
Spdl_Global_GreenDotsOK.xlsx
Spdl_Global_GreenLinesOK.xlsx
Spdl_Global_RedLinesOK.xlsx
Spdl_SpPSTH_BlackLinesOK.xlsx
Spdl_SpPSTH_BlueLinesOK.xlsx
Spdl_SpPSTH_GreenDotsOK.xlsx
Spdl_SpPSTH_GreenLinesOK.xlsx
Spdl_SpPSTH_RedLinesOK.xlsx
130 L1 neurons recorded
129 L1 neurons in the cross-registration
1170 Spdl recorded in total
SWR oscillations ana

In [34]:
# Assuming Osc and combined_dfOsc are already defined
combined_dfOscS1 = combined_dfOsc[(combined_dfOsc[f'{Osc}StartLocation'] == 'S1') & (combined_dfOsc[f'GlobalSpindle'] == 'Local')]
combined_dfOscS1

,Mice,Session,Session_Time,Unique_Unit,UnitNumber,UnitValue,Drug,SpdlStatut,SpdlStartLocation,GlobalSpindle,...,AUC_calciumBefore,AUC_calciumDuring,AUC_calciumAfter,SpikeActivityPreference,SpikeActivityBefore,SpikeActivityDuring,SpikeActivityAfter,Unit_ID,Session_ID,Spdl_ID
247,Purple,session12,NaN,593,0,2,Baseline,UnCoupled,S1,Local,...,0.000000,0.000000,0.000000,NaN,0.0,0.000000,0.000000,Purple593,Purplesession12,Purplesession128
413,Purple,session13,NaN,133,1,4,Baseline,UnCoupled,S1,Local,...,0.000000,0.000000,0.000000,NaN,0.0,0.000000,0.000000,Purple133,Purplesession13,Purplesession1320
423,Purple,session13,NaN,133,1,4,Baseline,UnCoupled,S1,Local,...,0.000000,0.000000,0.000000,NaN,0.0,0.000000,0.000000,Purple133,Purplesession13,Purplesession1330
424,Purple,session13,NaN,133,1,4,Baseline,UnCoupled,S1,Local,...,0.000045,0.000033,0.000025,NaN,0.0,0.000000,0.000000,Purple133,Purplesession13,Purplesession1331
841,Purple,session21,NaN,7,0,4,Baseline,UnCoupled,S1,Local,...,14.734006,8.706328,35.032887,During,0.0,0.007569,0.006705,Purple7,Purplesession21,Purplesession212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22809,ThreeColDotsOK,session34,NaN,0,0,30,Baseline,UnCoupled,S1,Local,...,0.000000,0.000000,0.000000,NaN,0.0,0.000000,0.000000,ThreeColDotsOK0,ThreeColDotsOKsession34,ThreeColDotsOKsession34277
22810,ThreeColDotsOK,session34,NaN,0,0,30,Baseline,UnCoupled,S1,Local,...,0.000000,0.000000,0.120093,After,0.0,0.000000,0.004647,ThreeColDotsOK0,ThreeColDotsOKsession34,ThreeColDotsOKsession34278
22811,ThreeColDotsOK,session34,NaN,0,0,30,Baseline,UnCoupled,S1,Local,...,18.982756,16.923513,15.087657,NaN,0.0,0.000000,0.000000,ThreeColDotsOK0,ThreeColDotsOKsession34,ThreeColDotsOKsession34279
22812,ThreeColDotsOK,session34,NaN,0,0,30,Baseline,UnCoupled,S1,Local,...,1.766371,1.627277,1.499135,NaN,0.0,0.000000,0.000000,ThreeColDotsOK0,ThreeColDotsOKsession34,ThreeColDotsOKsession34280


In [31]:
combined_dfOsc

,Mice,Session,Session_Time,Unique_Unit,UnitNumber,UnitValue,Drug,SpdlStatut,SpdlStartLocation,GlobalSpindle,...,AUC_calciumBefore,AUC_calciumDuring,AUC_calciumAfter,SpikeActivityPreference,SpikeActivityBefore,SpikeActivityDuring,SpikeActivityAfter,Unit_ID,Session_ID,Spdl_ID
0,Purple,session11,NaN,167,0,0,Baseline,Coupled,PFC,Local,...,28.159331,27.061543,26.006551,NaN,0.0,0.0,0.000000,Purple167,Purplesession11,Purplesession111
1,Purple,session11,NaN,167,0,0,Baseline,UnCoupled,PFC,Local,...,0.426369,0.386720,1.593947,After,0.0,0.0,0.002082,Purple167,Purplesession11,Purplesession112
2,Purple,session11,NaN,167,0,0,Baseline,UnCoupled,PFC,Local,...,0.620083,0.562419,0.510118,NaN,0.0,0.0,0.000000,Purple167,Purplesession11,Purplesession113
3,Purple,session11,NaN,167,0,0,Baseline,Coupled,PFC,Local,...,0.008727,0.008207,0.007718,NaN,0.0,0.0,0.000000,Purple167,Purplesession11,Purplesession114
4,Purple,session11,NaN,167,0,0,Baseline,Coupled,PFC,Local,...,0.000831,0.000793,0.000756,NaN,0.0,0.0,0.000000,Purple167,Purplesession11,Purplesession115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22809,ThreeColDotsOK,session34,NaN,0,0,30,Baseline,UnCoupled,S1,Local,...,0.000000,0.000000,0.000000,NaN,0.0,0.0,0.000000,ThreeColDotsOK0,ThreeColDotsOKsession34,ThreeColDotsOKsession34277
22810,ThreeColDotsOK,session34,NaN,0,0,30,Baseline,UnCoupled,S1,Local,...,0.000000,0.000000,0.120093,After,0.0,0.0,0.004647,ThreeColDotsOK0,ThreeColDotsOKsession34,ThreeColDotsOKsession34278
22811,ThreeColDotsOK,session34,NaN,0,0,30,Baseline,UnCoupled,S1,Local,...,18.982756,16.923513,15.087657,NaN,0.0,0.0,0.000000,ThreeColDotsOK0,ThreeColDotsOKsession34,ThreeColDotsOKsession34279
22812,ThreeColDotsOK,session34,NaN,0,0,30,Baseline,UnCoupled,S1,Local,...,1.766371,1.627277,1.499135,NaN,0.0,0.0,0.000000,ThreeColDotsOK0,ThreeColDotsOKsession34,ThreeColDotsOKsession34280


In [27]:
# Stats on Ca2+ imaging with miniscope and Vigilance States

#######################################################################################
                            # Define Experiment type #
#######################################################################################

#DrugExperiment=1 if CGP Experiment // DrugExperiment=0 if Baseline Experiment
DrugExperiment=1

AnalysisID='_AB_FINAL' #to identify this analysis from another

choosed_folder='VigSt_2024-07-22_18_21_32_AB_FINAL' if DrugExperiment else 'VigSt_2024-07-22_17_16_28_AB_FINAL'

desired_order = ['Wake','NREM', 'REM']   
#desired_order = ['Wake', 'N2', 'NREM', 'REM'] 

#######################################################################################
                                # Load packages #
#######################################################################################

import statsmodels.api as sm
import quantities as pq
import numpy as np
import math 
import neo
import json
from pathlib import Path
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
import pickle
import os
import statsmodels.api as sm
import statsmodels.formula.api as smf
from datetime import datetime
import shutil
from scipy.stats import ttest_ind
import statsmodels.api as sm
import re

import warnings
warnings.filterwarnings("ignore")

def extract_micename(index_value):
    match = re.match(r"([a-zA-Z]+)", index_value)
    return match.group(1) if match else index_value

def max_column_name(row):
    return row.idxmax()
    
def discrimination_index(df):
    #Index = (AUCa-AUCb)/(AUCa+AUCb)
    if 'REM' in df.columns and 'NREM' in df.columns :
        disc_index =(df['NREM']-df['REM'])/(df['NREM']+df['REM'])
        nan_rows = df['NREM'].isna() | df['REM'].isna()
        disc_index[nan_rows] = np.nan
    else:
        disc_index=np.full(len(df), np.nan)
    #min=df[['NREM', 'REM']].min(axis=1).abs()+1
    #NREMadj=df['NREM']#+min
    #REMadj=df['REM']#+min
    #disc_index=NREMadj/REMadj
    return disc_index

def normalize_row(row):
    max_col = row.idxmax()  # Find the column with the maximum value
    max_val = row[max_col]  # Get the maximum value
    return row / max_val   # Normalize the row by dividing by the maximum value    

########################################################################
        # SCRIPT 23AB_GrandAverages&Stats_for_VigilanceStates
########################################################################

# Specify the directory containing the Excel files
InitialDirectory = "//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/CGP/AB_Analysis" if DrugExperiment else f"//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording_ABmodified/AB_Analysis"
directory= f'{InitialDirectory}/{choosed_folder}'

# Get the current date and time
FolderNameSave=str(datetime.now())[:19]
FolderNameSave = FolderNameSave.replace(" ", "_").replace(".", "_").replace(":", "_")
destination_folder= f"//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/CGP/AB_Analysis/AVG_VigSt_{FolderNameSave}{AnalysisID}" if DrugExperiment else f"//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording_ABmodified/AB_Analysis/AVG_VigSt_{FolderNameSave}{AnalysisID}"
os.makedirs(destination_folder)
folder_to_save=Path(destination_folder)

# Copy the script file to the destination folder
source_script = "C:/Users/Manip2/SCRIPTS/CodePythonAudrey/CodePythonAurelie/HayLabAnalysis/python/23_24_GrandAverage&Stats_for_VigilanceStates_FullAuto.py"
destination_file_path = f"{destination_folder}/23_24_GrandAverage&Stats_for_VigilanceStates_FullAuto.txt"
shutil.copy(source_script, destination_file_path)

NrSubtypeList=['L2&3', 'L1']

for NrSubtype in NrSubtypeList:  

    # Initialize an empty list to store the dataframes
    dfs = []
    df= []
    df2=[]
    dfs2_per_sheet = {}
    dfs3 = []
    df3=[]
    dfs3_per_sheet = {}

    if NrSubtype=='L1':
        MiceList=['BlackLinesOK', 'BlueLinesOK', 'GreenDotsOK', 'GreenLinesOK', 'RedLinesOK']
    else:
        MiceList=['Purple', 'ThreeColDotsOK', 'ThreeBlueCrossesOK']
    
    nametofind='Global'
    nametofind2='CaCorr'
    nametofind3='SpCorr'

    # Recursively traverse the directory structure
    for root, _, files in os.walk(directory):
        for filename in files:
            # Check if the file is an Excel file and contains the specified name
            if filename.endswith('.xlsx') and nametofind in filename : 
                if any(name in filename for name in MiceList): 
                    # Construct the full path to the file
                    filepath = os.path.join(root, filename)
                    # Read the Excel file into a dataframe and append it to the list
                    df = pd.read_excel(filepath, index_col=0)      
                    dfs.append(df)
            if filename.endswith('.xlsx') and nametofind2 in filename: 
                if any(name in filename for name in MiceList): 
                    # Construct the full path to the file
                    filepath = os.path.join(root, filename)
                    # Read the Excel file into a dataframe and append it to the list
                    excel_data = pd.read_excel(filepath, sheet_name=None, index_col=0)           
                    for sheet_name, df2 in excel_data.items():
                        if len(df2)>0:
                            if sheet_name in dfs2_per_sheet:                                       
                                updated_matrix = pd.concat([dfs2_per_sheet[sheet_name], df2], axis=0)                    
                                dfs2_per_sheet[sheet_name] = updated_matrix    
                            else:
                                dfs2_per_sheet[sheet_name] = df2 
                    # Keep only correlation that occurs for the 3 vigilances states / the 2 Drugs
                    first_key = next(iter(dfs2_per_sheet))
                    common_columns = dfs2_per_sheet[first_key].columns
                    for df in dfs2_per_sheet.values():
                        common_columns = common_columns.intersection(df.columns)

                    common_indices = dfs2_per_sheet[first_key].index
                    for df in dfs2_per_sheet.values():
                        common_indices = common_indices.intersection(df.index)

                    filtered_df_dict2 = {name: df.loc[common_indices, common_columns] for name, df in dfs2_per_sheet.items()}
            if filename.endswith('.xlsx') and nametofind3 in filename: 
                if any(name in filename for name in MiceList): 
                    # Construct the full path to the file
                    filepath = os.path.join(root, filename)
                    # Read the Excel file into a dataframe and append it to the list
                    excel_data = pd.read_excel(filepath, sheet_name=None, index_col=0)           
                    for sheet_name, df3 in excel_data.items():
                        if len(df3)>0:
                            if sheet_name in dfs3_per_sheet:   
                                updated_matrix = pd.concat((dfs3_per_sheet[sheet_name],df3), axis=0)                
                                dfs3_per_sheet[sheet_name] = updated_matrix                    
                            else:                    
                                dfs3_per_sheet[sheet_name] = df3 #one average trace per unique unit, len(df3)==nb unit recorded for that mouse
                    # Keep only correlation that occurs for the 3 vigilances states / the 2 Drugs
                    first_key = next(iter(dfs3_per_sheet))
                    common_columns = dfs3_per_sheet[first_key].columns
                    for df in dfs3_per_sheet.values():
                        common_columns = common_columns.intersection(df.columns)

                    common_indices = dfs3_per_sheet[first_key].index
                    for df in dfs3_per_sheet.values():
                        common_indices = common_indices.intersection(df.index)

                    filtered_df_dict3 = {name: df.loc[common_indices, common_columns] for name, df in dfs3_per_sheet.items()}

    # Concatenate all dataframes into a single dataframe
    combined_df = pd.concat(dfs, ignore_index=True)

    # Remove non defined Unique Units 
    combined_df = combined_df[combined_df['Unique_Unit'] != '[]']
    combined_df = combined_df.dropna(subset=['Unique_Unit'])

    combined_df['Unique_Unit'] = combined_df['Unique_Unit'].astype(int).astype(str)
    combined_df['UnitNumber'] = combined_df['UnitNumber'].astype(str)
    combined_df['UnitValue'] = combined_df['UnitValue'].astype(str)

    combined_df['Unit_ID'] = combined_df['Mice'] + combined_df['Unique_Unit']
    
    combined_df['NormalizedAUC_calcium'] = combined_df['AUC_calcium'] / combined_df['DurationSubstate']

    combined_df['Substate_ID'] = combined_df['Mice'] + combined_df['Session'] + combined_df['Substate'] + combined_df['SubstateNumber'].astype(str)
    combined_df['Session_ID'] = combined_df['Mice'] + combined_df['Session'].astype(str)

    # Save big dataset for stats

    filenameOut = f'{folder_to_save}/{NrSubtype}_VigSt_Global.xlsx'
    writer = pd.ExcelWriter(filenameOut)
    combined_df.to_excel(writer)
    writer.close()

    # Save the Ca correlation matrix  

    file_path = f'{folder_to_save}/{NrSubtype}_VigSt_CaCorr.xlsx'
    with pd.ExcelWriter(file_path) as writer:
        for sheet_name, df in dfs2_per_sheet.items():
            df.to_excel(writer, sheet_name=sheet_name, index=True, header=True)

    # Save the Sp correlation matrix  

    file_path = f'{folder_to_save}/{NrSubtype}_VigSt_SpCorr.xlsx'
    with pd.ExcelWriter(file_path) as writer:
        for sheet_name, df in dfs3_per_sheet.items():
            df.to_excel(writer, sheet_name=sheet_name, index=True, header=True)

########################################################################
            # SCRIPT 24AB_Load&Stats_for_VigilanceStates
########################################################################

AllProportionVigStates=pd.DataFrame()
AllDurationVigStates=pd.DataFrame()
AllTotDurationVigStates=pd.DataFrame()

for NrSubtype in NrSubtypeList:

    analysisfile='VigSt_Global'
    combined_dfO = pd.read_excel(f'{folder_to_save}/{NrSubtype}_{analysisfile}.xlsx', index_col=0)
    
    combined_dfCa = {}
    analysisfileCa='VigSt_CaCorr'
    excel_data = pd.read_excel(f'{folder_to_save}/{NrSubtype}_{analysisfileCa}.xlsx', sheet_name=None, index_col=0)           
    for sheet_name, df in excel_data.items():
        if len(df)>0:
            combined_dfCa[sheet_name] = df  

    combined_dfSp = {}
    analysisfileSp='VigSt_SpCorr'
    excel_data = pd.read_excel(f'{folder_to_save}/{NrSubtype}_{analysisfileSp}.xlsx', sheet_name=None, index_col=0)           
    for sheet_name, df in excel_data.items():
        if len(df)>0:
            combined_dfSp[sheet_name] = df  

    ######################
    # CHOOSE OPTIONS
    ######################

    # MINIMUM VIG STATES DURATION #
    combined_df = combined_dfO[combined_dfO['DurationSubstate'] >= 20] 
    
    Drugs= ['Baseline', 'CGP'] if DrugExperiment else ['Baseline']

    # NO LOW FIRING RATE #
    #combined_df = combined_df[combined_df['Avg_SpikeActivityHz'] >= 0.05] 

    #######################
    # Propreties VigStates
    #######################

    filenameOut = f'{folder_to_save}/VigStPropreties.xlsx'
    writer = pd.ExcelWriter(filenameOut)

    combined_df2 = combined_dfO.drop_duplicates(subset='Substate_ID', keep='first')

    ProportionVigStates = combined_df2.pivot_table(index='Session_ID', columns=[combined_df2['Drug'], combined_df2['Substate']], values='DurationSubstate', aggfunc='sum', fill_value=None)
    try: ProportionVigStates = ProportionVigStates[desired_order]
    except: pass
    ProportionVigStates=ProportionVigStates.div(ProportionVigStates.sum(axis=1), axis=0)*100
    AllProportionVigStates=pd.concat([AllProportionVigStates, ProportionVigStates], axis=0)

    DurationVigStates = combined_df2.pivot_table(index='Session_ID', columns=[combined_df2['Drug'], combined_df2['Substate']], values='DurationSubstate', aggfunc='mean', fill_value=None)
    try: DurationVigStates = DurationVigStates[desired_order]
    except: pass        
    AllDurationVigStates=pd.concat([AllDurationVigStates, DurationVigStates], axis=0)

    TotDurationVigStates = combined_df2.pivot_table(index='Session_ID', columns=[combined_df2['Drug'], combined_df2['Substate']], values='DurationSubstate', aggfunc='sum', fill_value=None)
    try: TotDurationVigStates = TotDurationVigStates[desired_order]
    except: pass
    AllTotDurationVigStates=pd.concat([AllTotDurationVigStates, TotDurationVigStates], axis=0)

AllProportionVigStates.to_excel(writer, sheet_name=f'ProportionVigStates')        
AllDurationVigStates.to_excel(writer, sheet_name=f'MeanEpisodeDurations')
AllTotDurationVigStates.to_excel(writer, sheet_name=f'TotalEpisodeDurations')
writer.close()
     